In [29]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('restaurant.db')
cur = conn.cursor()

try:
    cur.execute("DROP TABLE IF EXISTS MenuItem;")
    conn.commit()
    print("Table 'MenuItem' dropped successfully.")
except Exception as e:
    print(f"Error dropping table 'MenuItem': {e}")

try:
    cur.execute("DROP TABLE IF EXISTS Guest;")
    conn.commit()
    print("Table 'Guest' dropped successfully.")
except Exception as e:
    print(f"Error dropping table 'Guest': {e}")

try:
    cur.execute("DROP TABLE IF EXISTS Order1;")
    conn.commit()
    print("Table 'Guest' dropped successfully.")
except Exception as e:
    print(f"Error dropping table 'Order1': {e}")

# SQL commands to create tables and insert data
sql_commands = """
CREATE TABLE MenuItem (
    mid INT,
    mname VARCHAR(255),
    mtype VARCHAR(255),
    price DECIMAL(10, 2),
    PRIMARY KEY (mid)
);

CREATE TABLE Guest (
    gid INT,
    tableid INT,
    PRIMARY KEY (gid)
);

CREATE TABLE Order1 (
    oid INT,
    gid INT,
    mid INT,
    quantity INT,
    date DATE,
    PRIMARY KEY (oid),
    FOREIGN KEY (gid) REFERENCES Guest(gid),
    FOREIGN KEY (mid) REFERENCES MenuItem(mid)
);

INSERT INTO MenuItem (mid, mname, mtype, price) 
VALUES (1, 'Burger', 'Main Course', 9), (2, 'Salad', 'Appetizer', 5), (3, 'Pizza', 'Main Course', 12), (4, 'Fries', 'Side Dish', 3), (5, 'Ice Cream', 'Dessert', 4),
    (6, 'Pasta', 'Main Course', 10), (7, 'Soup', 'Appetizer', 6), (8, 'Sandwich', 'Main Course', 8), (9, 'Cake', 'Dessert', 7), (10, 'Steak', 'Main Course', 15);

INSERT INTO Guest (gid, tableid) 
VALUES (1, 10), (2, 10), (3, 1), (6, 1), (4, 2), (7, 2), (5, 3), (8, 6), (9, 4), (10, 5);

INSERT INTO Order1 (oid, gid, mid, quantity, date) 
VALUES (1, 1, 1, 3, '2022-01-01'), (2, 2, 2, 3, '2022-01-01'), (3, 3, 3, 1, '2022-01-03'), (4, 4, 4, 1, '2022-01-04'),
 (5, 5, 5, 1, '2022-01-05'), (6, 6, 6, 1, '2022-01-03'), (7, 7, 7, 2, '2022-01-04'), (8, 8, 8, 1, '2022-01-08'),(9, 9, 4, 2, '2022-01-09'), (10, 10, 3, 3, '2022-01-10'),
 (11, 1, 1, 3, '2022-01-02'), (12, 2, 2, 3, '2022-01-02'), (13, 3, 3, 2, '2022-01-04'), (14, 6, 6, 3, '2022-01-04');
"""

# Execute the SQL commands
cur.executescript(sql_commands)



Table 'MenuItem' dropped successfully.
Table 'Guest' dropped successfully.
Table 'Guest' dropped successfully.


In [9]:
print("mid, mname, mtype, price")
cur.execute("SELECT * FROM MenuItem")
rows = cur.fetchall()
for row in rows:
    print(row)

mid, mname, mtype, price
(1, 'Burger', 'Main Course', 9)
(2, 'Salad', 'Appetizer', 5)
(3, 'Pizza', 'Main Course', 12)
(4, 'Fries', 'Side Dish', 3)
(5, 'Ice Cream', 'Dessert', 4)
(6, 'Pasta', 'Main Course', 10)
(7, 'Soup', 'Appetizer', 6)
(8, 'Sandwich', 'Main Course', 8)
(9, 'Cake', 'Dessert', 7)
(10, 'Steak', 'Main Course', 15)


In [10]:
print("gid, tableid")
cur.execute("SELECT * FROM Guest")
rows = cur.fetchall()
for row in rows:
    print(row)

gid, tableid
(1, 10)
(2, 10)
(3, 1)
(6, 1)
(4, 2)
(7, 2)
(5, 3)
(8, 6)
(9, 4)
(10, 5)


In [11]:
print("oid, gid, mid, quantity, date")
cur.execute("SELECT * FROM Order1")
rows = cur.fetchall()
for row in rows:
    print(row)

oid, gid, mid, quantity, date
(1, 1, 1, 2, '2022-01-01')
(2, 2, 2, 1, '2022-01-01')
(3, 3, 3, 3, '2022-01-03')
(4, 4, 4, 2, '2022-01-04')
(5, 5, 5, 1, '2022-01-05')
(6, 6, 6, 3, '2022-01-03')
(7, 7, 7, 2, '2022-01-04')
(8, 8, 8, 1, '2022-01-08')
(9, 9, 4, 2, '2022-01-09')
(10, 10, 3, 3, '2022-01-10')
(11, 1, 1, 2, '2022-01-02')
(12, 2, 2, 1, '2022-01-02')
(13, 3, 3, 3, '2022-01-04')
(14, 6, 6, 3, '2022-01-04')


In [12]:

# SQL query to fetch data
sql_query = """
WITH GuestOrder1Count AS (
    SELECT o.mid, o.gid, COUNT(*) AS Order1_count
    FROM Order1 o
    GROUP BY o.mid, o.gid
),
MenuItemGuestCount AS (
    SELECT mid, COUNT(gid) AS guest_count
    FROM GuestOrder1Count
    WHERE Order1_count = 1
    GROUP BY mid
)
SELECT mid, guest_count
FROM MenuItemGuestCount
ORDER BY guest_count DESC

"""

# Execute the query
cur.execute(sql_query)
# Fetch and print the results
results = cur.fetchall()
for row in results:
    print(row)



(4, 2)
(3, 1)
(5, 1)
(7, 1)
(8, 1)


In [6]:
sql_query1 = """
WITH GuestMaxPrice AS (
    SELECT 
        g.gid,
        g.tableid,
        o.date,
        MAX(m.price) AS max_price
    FROM 
        Guest g
        JOIN Order1 o ON g.gid = o.gid
        JOIN MenuItem m ON o.mid = m.mid
    GROUP BY 
        g.gid, g.tableid, o.date
),

TableMaxPrice AS (
    SELECT 
        g.tableid,
        o.date,
        MAX(m.price) AS max_price
    FROM 
        Guest g
        JOIN Order1 o ON g.gid = o.gid
        JOIN MenuItem m ON o.mid = m.mid
    GROUP BY 
        g.tableid, o.date
),

ConsistentMaxPriceGuests AS (
    SELECT 
        gmp.gid,
        gmp.tableid,
        gmp.date
    FROM 
        GuestMaxPrice gmp
        JOIN TableMaxPrice tmp ON gmp.tableid = tmp.tableid AND gmp.date = tmp.date
    WHERE 
        gmp.max_price = tmp.max_price
),

EligibleGuests AS (
    SELECT 
        gid
    FROM 
        ConsistentMaxPriceGuests
    GROUP BY 
        gid
    HAVING 
        COUNT(*) = (SELECT COUNT(*) FROM GuestMaxPrice WHERE gid = ConsistentMaxPriceGuests.gid)
)

SELECT 
    g.gid,
    g.tableid
FROM 
    Guest g
JOIN 
    EligibleGuests eg ON g.gid = eg.gid;


"""

cur.execute(sql_query1)


results = cur.fetchall()
for row in results:
    print(row)


(1, 10)
(3, 1)
(5, 3)
(7, 2)
(8, 6)
(9, 4)
(10, 5)


In [31]:
sql_query2 = """
WITH GuestVisitAmounts AS (
    SELECT 
        g.gid,
        g.tableid,
        o.date,
        SUM(o.quantity * m.price) AS total_amount
    FROM 
        Guest g
        JOIN Order1 o ON g.gid = o.gid
        JOIN MenuItem m ON o.mid = m.mid
    GROUP BY 
        g.gid, g.tableid, o.date
),

GuestPairs AS (
    SELECT 
        gva1.gid AS gid1,
        gva2.gid AS gid2,
        gva1.tableid,
        gva1.date,
        gva1.total_amount AS amount1,
        gva2.total_amount AS amount2,
        gva1.total_amount + gva2.total_amount AS combined_amount
    FROM 
        GuestVisitAmounts gva1
        JOIN GuestVisitAmounts gva2 
            ON gva1.tableid = gva2.tableid 
            AND gva1.date = gva2.date 
            AND gva1.gid < gva2.gid
),

TotalAmountsByPair AS (
    SELECT 
        gid1, 
        gid2, 
        SUM(combined_amount) AS total_combined_amount
    FROM 
        GuestPairs
    GROUP BY 
        gid1, gid2
),

MaxTotalAmountPair AS (
    SELECT 
        gid1, 
        gid2, 
        total_combined_amount
    FROM 
        TotalAmountsByPair
    ORDER BY 
        total_combined_amount DESC
    LIMIT 1
)

SELECT 
    mtap.gid1, 
    mtap.gid2, 
    mtap.total_combined_amount
FROM 
    MaxTotalAmountPair mtap;



"""

cur.execute(sql_query2)


results = cur.fetchall()
for row in results:
    print(row)

(1, 2, 84)


In [ ]:
# Close the cursor and connection
cur.close()
conn.close()